In [1]:
import csv
import json
import re
import pandas as pd
import numpy as np


In [5]:
routes = pd.read_csv('Skiturer Sunnmøre - Turer.csv')

columns = {
  'Fjell': 'mountain_name',
  'Himmelretning (topp til bunn)': 'aspect',
  'Område': 'area',
  'Rute': 'route_name',
  'Høyeste startsted': 'max_start_altitude',
  'Løsneområder': 'avalanche_start_zones',
  'Terrengfeller': 'terrain_traps',
  'KAST': 'kast_score'
}

routes = routes.rename(columns=columns)
routes = routes.dropna(how='all', axis='columns')
routes = routes.fillna('')

TypeError: 'float' object is not subscriptable

In [3]:
routes_dict = routes.to_dict(orient='records')

for route in routes_dict:
      if route['geoJSON']:
        route['geoJSON'] = json.loads(route['geoJSON'])

      if route['aspect']:
        route['aspect'] = route['aspect'].split(' ')

      if isinstance(route['avalanche_start_zones'], str):
        elevation_spans = route['avalanche_start_zones'].replace(',', '.').split('.')
        elevation_spans = [tuple(span.strip().split(':')) for span in elevation_spans]
        route['start_zones'] = elevation_spans
routes_dict

[{'fjell_id': 7.0,
  'mountain_name': 'Eidskyrkja',
  'Toppunkt': '',
  'area': '',
  'route_name': 'Fra Skinnviksætra',
  'aspect': 'N',
  'max_start_altitude': 390.0,
  'avalanche_start_zones': '',
  'Max bratthet': '',
  'terrain_traps': '',
  'Eksponering': '',
  'Vanskelighetsgrad': '',
  'geoJSON': '',
  'Kan gjøres trygt': 'Bratt i renna',
  'start_zones': [('',)]},
 {'fjell_id': 7.0,
  'mountain_name': 'Eidskyrkja',
  'Toppunkt': '',
  'area': '',
  'route_name': 'Fra Grøndalen',
  'aspect': '',
  'max_start_altitude': 420.0,
  'avalanche_start_zones': '',
  'Max bratthet': '',
  'terrain_traps': '',
  'Eksponering': '',
  'Vanskelighetsgrad': '',
  'geoJSON': '',
  'Kan gjøres trygt': '',
  'start_zones': [('',)]},
 {'fjell_id': 11.0,
  'mountain_name': 'Fingeren',
  'Toppunkt': '',
  'area': '',
  'route_name': 'Normalveien',
  'aspect': 'SØ',
  'max_start_altitude': 360.0,
  'avalanche_start_zones': '',
  'Max bratthet': '',
  'terrain_traps': '',
  'Eksponering': '',
  'Van

In [4]:
with open('routes.json', 'w') as file:
    file.write(json.dumps(routes_dict))
